In [1]:
# Nate Bennett

# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash

from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDPythonScript import CRUD



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "aacuser_password"
HOST = 'nv-desktop-services.apporto.com'
PORT = 30933
DB = 'AAC'
COL = 'animals'

crud = CRUD(DB, COL, username, password, HOST, PORT)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(crud.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px', 'margin': '0 auto'}),
        style={'text-align': 'center'}
    ),
    html.H2("Nate Bennett"),
    html.Hr(),
    dcc.RadioItems(
        id='filter_type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'},
        ],
        value='Reset',
        labelStyle={'display': 'inline'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,  # You can set the number of rows per page here
        row_selectable="single",  # Enable single-row selection
        style_table={'height': '400px', 'overflowY': 'auto'}  # Add styling for the table

    ),
    html.Br(),
    html.Hr(),
    html.Div(
        style={'display': 'inline-block', 'width': '50%'},
        children=[
            html.Div(
                id='graph-id',  # This is a div element with the id 'graph-id'
            ),
        ]
    ),
    html.Div(
        style={'display': 'inline-block', 'width': '50%'},
        children=[
            html.Div(
                id='map-id',
                style={'height': '500px'}  # Set the height of the map
            )
        ]
    )




])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id','data'),
    [Input('filter_type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)
    if filter_type == 'Reset':
        # Reset button clicked, show all data
        query={}
    elif filter_type == 'Water Rescue':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'breed': {'$in': ['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland/Labrador Retriever',
                                   'Newfoundland Mix', 'Newfoundland/Great Pyrenees',
                                   'Newfoundland/Australian Cattle Dog', 'Labrador Retriever/Newfoundland']}},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                   'Rottweiler', 'Siberian Husky']}},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }
    elif filter_type == 'Disaster Rescue or Individual Tracking':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever',
                                   'Bloodhound', 'Rottweiler']}},
                {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
            ]
        }
        
    filtered_df = pd.DataFrame.from_records(crud.read(query))
    
    # If '_id' exists in the columns, drop it
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)



    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    # Create a DataFrame from the displayed data
    dff = pd.DataFrame.from_dict(viewData)
    # Get breed names and their counts for the pie chart
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure=px.pie(
                data_frame=dff, 
                values=values, 
                names=names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                
            )
        )
    ]
    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    if index is None or len(index) == 0:
        return dl.Map(
            style={'width': '100%', 'height': '100%'},
            center=[30.75, -97.48], zoom=10,  # Default to Austin, TX
            children=[
                dl.TileLayer(id="base-layer-id"),
            ]
        )
    else:
        row = index[0]
        return dl.Map(
            style={'width': '100%', 'height': '100%'},
            center=[df.iloc[row, 13], df.iloc[row, 14]], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[df.iloc[row, 13], df.iloc[row, 14]],
                    children=[
                        dl.Tooltip(df.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(df.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:10784/
